### Manipulation of Peaks de novo results of ETNP 2017 P2 samples LC-MS/MS data using python.

Starting with:

    Peaks de novo results (.csv) of PTM-optimized database searches

Goal:

    Files with stripped (no PTMs) peptide lists and
    Columns with #'s of each modification in every sequence
    Column with stripped peptide lengths (# amino acids)
    
### To use for a different file:

#### 1. Change the input file name in *IN 4*
#### 2. Use 'find + replace' (Esc + F) to replace the running # (e.g., 233) for another
#### 3. Update the NAAF factor calculated in *IN 6* into *IN 7*

We don't have technical duplicates here, sadly, unlike the MED4 Pro samples. I exported PeaksDN search results CSVs into my ETNP 2017 git repo:

In [1]:
cd /home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/

/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt


In [2]:
# LIBRARIES
#import pandas library for working with tabular data
import os
os.getcwd()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kde
#import regular expresson (regex)
import re
#check pandas version
pd.__version__

'1.0.5'

## 378: 100 m +P filtered on 0.3 um GF-75

In [3]:
# read the CSV into a dataframe using the pandas read_csv function
peaks378_50 = pd.read_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-378-100m-trap_DN50.csv")

# also make a dataframe but keep only peptides >80% ALC
peaks378_80 = peaks378_50.loc[peaks378_50['ALC (%)'] >= 80].copy()

# how many de novo sequence candidates >50% ALC?
print("# redundant Peaks peptides >50% ALC in dataframe", len(peaks378_50))
# how many de novo sequence candidates >80ALC?
print("# redundant Peaks peptides >80% ALC in dataframe", len(peaks378_80))

#look at the dataframe
peaks378_50.head()

# redundant Peaks peptides >50% ALC in dataframe 1590
# redundant Peaks peptides >80% ALC in dataframe 465


,Fraction,Scan,Source File,Peptide,Tag Length,ALC (%),length,m/z,z,RT,Area,Mass,ppm,PTM,local confidence (%),tag (>=0%),mode
0,4,4564,20181214_378_etnp2017_100m_trap.raw,VVEEAPK,7,99,7,386.2155,2,17.85,800000.0,770.4174,-1.1,NaN,99 99 100 100 99 99 99,VVEEAPK,CID
1,4,4574,20181214_378_etnp2017_100m_trap.raw,SLEAESR,7,98,7,396.1982,2,17.88,513000.0,790.3821,-0.4,NaN,98 99 100 99 99 98 96,SLEAESR,CID
2,4,4982,20181214_378_etnp2017_100m_trap.raw,EEDLER,6,98,6,395.6826,2,19.14,245000.0,789.3504,0.3,NaN,100 100 98 99 99 95,EEDLER,CID
3,4,3812,20181214_378_etnp2017_100m_trap.raw,TEEAVEEK,8,98,8,467.7217,2,15.60,232000.0,933.4291,-0.2,NaN,99 100 100 99 97 99 99 90,TEEAVEEK,CID
4,4,3992,20181214_378_etnp2017_100m_trap.raw,ELAGNAAR,8,97,8,401.2144,2,16.13,485000.0,800.4140,0.3,NaN,98 96 99 97 98 99 99 98,ELAGNAAR,CID


In [4]:
# use a count function to enumerate the # of A's (alanines) in each peptide
peaks378_50['A'] = peaks378_50['Peptide'].str.count("A")

# use a count function to enumerate the # of C's (cysteines) in each peptide
peaks378_50['C'] = peaks378_50['Peptide'].str.count("C")

# use a count function to enumerate the # of D's (aspartic acids) in each peptide
peaks378_50['D'] = peaks378_50['Peptide'].str.count("D")

# use a count function to enumerate the # of E's (glutamic acids) in each peptide
peaks378_50['E'] = peaks378_50['Peptide'].str.count("E")

# use a count function to enumerate the # of F's (phenylalanines) in each peptide
peaks378_50['F'] = peaks378_50['Peptide'].str.count("F")

# use a count function to enumerate the # of G's (glycines) in each peptide
peaks378_50['G'] = peaks378_50['Peptide'].str.count("G")

# use a count function to enumerate the # of H's (histidines) in each peptide
peaks378_50['H'] = peaks378_50['Peptide'].str.count("H")

# use a count function to enumerate the # of I's (isoleucines) in each peptide
# in peaks378_50 output, there will be no isoleucines (they're lumped in with leucines)
peaks378_50['I'] = peaks378_50['Peptide'].str.count("I")

# use a count function to enumerate the # of K's (lysines) in each peptide
peaks378_50['K'] = peaks378_50['Peptide'].str.count("K")

# use a count function to enumerate the # of L's (leucines) in each peptide
# also these include the isoleucines
peaks378_50['I/L'] = peaks378_50['Peptide'].str.count("L")

# use a count function to enumerate the # of M's (methionines) in each peptide
peaks378_50['M'] = peaks378_50['Peptide'].str.count("M")

# use a count function to enumerate the # of N's (asparagines) in each peptide
peaks378_50['N'] = peaks378_50['Peptide'].str.count("N")

# use a count function to enumerate the # of P's ([prolines]) in each peptide
peaks378_50['P'] = peaks378_50['Peptide'].str.count("P")

# use a count function to enumerate the # of Q's (glutamines) in each peptide
peaks378_50['Q'] = peaks378_50['Peptide'].str.count("Q")

# use a count function to enumerate the # of R's (arginines) in each peptide
peaks378_50['R'] = peaks378_50['Peptide'].str.count("R")

# use a count function to enumerate the # of S's (serines) in each peptide
peaks378_50['S'] = peaks378_50['Peptide'].str.count("S")

# use a count function to enumerate the # of T's (threonines) in each peptide
peaks378_50['T'] = peaks378_50['Peptide'].str.count("T")

# use a count function to enumerate the # of V's (valines) in each peptide
peaks378_50['V'] = peaks378_50['Peptide'].str.count("V")

# use a count function to enumerate the # of W's (tryptophans) in each peptide
peaks378_50['W'] = peaks378_50['Peptide'].str.count("W")

# use a count function to enumerate the # of Y's (tyrosines) in each peptide
peaks378_50['Y'] = peaks378_50['Peptide'].str.count("Y")

# use a count function to enumerate the # of carbamidomethylated C's in each peptide
peaks378_50['c-carb'] = peaks378_50['Peptide'].str.count("57.02")

# use a count function to enumerate the # of oxidized M's in each peptide
peaks378_50['m-oxid'] = peaks378_50['Peptide'].apply(lambda x: x.count('M(+15.99)'))

# use a lamba function to enumerate the # of deamidated N's in each peptide
peaks378_50['n-deam'] = peaks378_50['Peptide'].apply(lambda x: x.count('N(+.98)'))

# use a count function to enumerate the # of deamidated Q's in each peptide
peaks378_50['q-deam'] = peaks378_50['Peptide'].apply(lambda x: x.count('Q(+.98)'))

# use a count function to enumerate the # of hydroxylated K's in each peptide
peaks378_50['k-hydr'] = peaks378_50['Peptide'].apply(lambda x: x.count('K(+15.99)'))

# use a count function to enumerate the # of hydroxylated P's in each peptide
peaks378_50['p-hydr'] = peaks378_50['Peptide'].apply(lambda x: x.count('P(+15.99)'))

# use a count function to enumerate the # of methylated R's in each peptide
peaks378_50['r-meth'] = peaks378_50['Peptide'].apply(lambda x: x.count('R(+14.02)'))

# create a column with 'stripped' peptide sequences using strip
peaks378_50['stripped peptide'] = peaks378_50['Peptide'].str.replace(r"\(.*\)","")

# add a column with the stripped peptide length (number of AAs)
peaks378_50['stripped length'] = peaks378_50['stripped peptide'].apply(len)

# total the number of modifications in sequence
peaks378_50['ptm-total'] = peaks378_50['c-carb'] + peaks378_50['m-oxid'] + peaks378_50['n-deam'] + peaks378_50['q-deam'] + peaks378_50['k-hydr'] + peaks378_50['p-hydr'] + peaks378_50['r-meth']

# calculate NAAF numerator for each peptide k
peaks378_50['NAAF num.'] = peaks378_50['Area'] / peaks378_50['stripped length']

# write modified dataframe to new csv file
peaks378_50.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-378-100m-trap_PTMopt_DN50.csv")


# check out the results
peaks378_50.head()

,Fraction,Scan,Source File,Peptide,Tag Length,ALC (%),length,m/z,z,RT,...,m-oxid,n-deam,q-deam,k-hydr,p-hydr,r-meth,stripped peptide,stripped length,ptm-total,NAAF num.
0,4,4564,20181214_378_etnp2017_100m_trap.raw,VVEEAPK,7,99,7,386.2155,2,17.85,...,0,0,0,0,0,0,VVEEAPK,7,0,114285.714286
1,4,4574,20181214_378_etnp2017_100m_trap.raw,SLEAESR,7,98,7,396.1982,2,17.88,...,0,0,0,0,0,0,SLEAESR,7,0,73285.714286
2,4,4982,20181214_378_etnp2017_100m_trap.raw,EEDLER,6,98,6,395.6826,2,19.14,...,0,0,0,0,0,0,EEDLER,6,0,40833.333333
3,4,3812,20181214_378_etnp2017_100m_trap.raw,TEEAVEEK,8,98,8,467.7217,2,15.60,...,0,0,0,0,0,0,TEEAVEEK,8,0,29000.000000
4,4,3992,20181214_378_etnp2017_100m_trap.raw,ELAGNAAR,8,97,8,401.2144,2,16.13,...,0,0,0,0,0,0,ELAGNAAR,8,0,60625.000000


In [5]:
# made a new dataframe that contains the sums of certain columns in the modified
# peptide dataframe above (for >50% ALC)

index = ['sample total']

data = {'A': peaks378_50['A'].sum(),
        'C': peaks378_50['C'].sum(),
        'D': peaks378_50['D'].sum(),
        'E': peaks378_50['E'].sum(),
        'F': peaks378_50['F'].sum(),
        'G': peaks378_50['G'].sum(),
        'H': peaks378_50['H'].sum(),
        'I': peaks378_50['I'].sum(),
        'K': peaks378_50['K'].sum(),
        'I/L': peaks378_50['I/L'].sum(),
        'M': peaks378_50['M'].sum(),
        'N': peaks378_50['N'].sum(),
        'P': peaks378_50['P'].sum(),
        'Q': peaks378_50['Q'].sum(),
        'R': peaks378_50['R'].sum(),
        'S': peaks378_50['S'].sum(),
        'T': peaks378_50['T'].sum(),
        'V': peaks378_50['V'].sum(),
        'W': peaks378_50['W'].sum(),
        'Y': peaks378_50['Y'].sum(),
        'c-carb': peaks378_50['c-carb'].sum(),
        'm-oxid': peaks378_50['m-oxid'].sum(),
        'n-deam': peaks378_50['n-deam'].sum(),
        'q-deam': peaks378_50['q-deam'].sum(),
        'k-hydr': peaks378_50['k-hydr'].sum(),
        'p-hydr': peaks378_50['p-hydr'].sum(),
        'r-meth': peaks378_50['r-meth'].sum(),
        'Total area': peaks378_50['Area'].sum(),
        'Total length': peaks378_50['stripped length'].sum()
       }

totalpeaks378_50 = pd.DataFrame(data, columns=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'I/L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', 'c-carb', 'm-oxid', 'n-deam', 'q-deam', 'k-hydr', 'p-hydr', 'r-meth', 'Total area', 'Total length'], index=index)

# calculate percentage of C's with carb (should be 1.0)
totalpeaks378_50['% C w/ carb'] = totalpeaks378_50['c-carb'] / totalpeaks378_50['C'] 

# calculate percentage of M's that are oxidized
totalpeaks378_50['% M w/ oxid'] = totalpeaks378_50['m-oxid'] / totalpeaks378_50['M'] 

# calculate percentage of N's that are deamidated
totalpeaks378_50['% N w/ deam'] = totalpeaks378_50['n-deam'] / totalpeaks378_50['N'] 

# calculate percentage of Q's that are deamidated
totalpeaks378_50['% Q w/ deam'] = totalpeaks378_50['q-deam'] / totalpeaks378_50['Q'] 

# calculate percentage of K's that are hydroxylated
totalpeaks378_50['% K w/ hydr'] = totalpeaks378_50['k-hydr'] / totalpeaks378_50['K'] 

# calculate percentage of P's that are hydroxylated
totalpeaks378_50['% P w/ hydr'] = totalpeaks378_50['p-hydr'] / totalpeaks378_50['K'] 

# calculate percentage of R's that are methylated
totalpeaks378_50['% R w/ meth'] = totalpeaks378_50['r-meth'] / totalpeaks378_50['R'] 

# calculate NAAF denominator for all peptides in dataset i
totalpeaks378_50['NAAF denom.'] = totalpeaks378_50['Total area'] / totalpeaks378_50['Total length']

# write modified dataframe to new txt file
totalpeaks378_50.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-378-100m-trap_PTMopt_DN50_totals.csv")

totalpeaks378_50.head()

,A,C,D,E,F,G,H,I,K,I/L,...,Total area,Total length,% C w/ carb,% M w/ oxid,% N w/ deam,% Q w/ deam,% K w/ hydr,% P w/ hydr,% R w/ meth,NAAF denom.
sample total,1102,231,651,1071,266,546,234,0,1434,838,...,1.960158e+09,10728,1.0,0.49505,0.255269,0.052747,0.154114,0.283821,0.342792,182714.230543


In [15]:
# use the calculated NAAF factor (in totalpeaks378 dataframe, above) to caluclate the NAAF 
# NAAF: normalized normalized area abundance factor

# don't have to worry here about DECOY hits messing with Area totals
# but we will with Comet results

NAAF50 = 182714.230543

# use NAAF >50% ALC to get NAAF
peaks378_50['NAAF factor'] = (peaks378_50['NAAF num.'])/NAAF50

# make a dataframe that contains only what we need: sequences, AAs, PTMs
peaks378_AA50 = peaks378_50[['stripped peptide', 'NAAF factor', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'K', 'I/L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', 'c-carb', 'm-oxid', 'n-deam', 'q-deam', 'k-hydr', 'p-hydr', 'r-meth']].copy()

# multiply the NAAF50 factor by the AAs to normalize its abundance by peak area and peptide length

peaks378_AA50['A-NAAF50'] = peaks378_AA50['A'] * peaks378_50['NAAF factor']
peaks378_AA50['C-NAAF50'] = peaks378_AA50['C'] * peaks378_50['NAAF factor']
peaks378_AA50['D-NAAF50'] = peaks378_AA50['D'] * peaks378_50['NAAF factor']
peaks378_AA50['E-NAAF50'] = peaks378_AA50['E'] * peaks378_50['NAAF factor']
peaks378_AA50['F-NAAF50'] = peaks378_AA50['F'] * peaks378_50['NAAF factor']
peaks378_AA50['G-NAAF50'] = peaks378_AA50['G'] * peaks378_50['NAAF factor']
peaks378_AA50['H-NAAF50'] = peaks378_AA50['H'] * peaks378_50['NAAF factor']
peaks378_AA50['K-NAAF50'] = peaks378_AA50['K'] * peaks378_50['NAAF factor']
peaks378_AA50['I/L-NAAF50'] = peaks378_AA50['I/L'] * peaks378_50['NAAF factor']
peaks378_AA50['M-NAAF50'] = peaks378_AA50['M'] * peaks378_50['NAAF factor']
peaks378_AA50['N-NAAF50'] = peaks378_AA50['N'] * peaks378_50['NAAF factor']
peaks378_AA50['P-NAAF50'] = peaks378_AA50['P'] * peaks378_50['NAAF factor']
peaks378_AA50['Q-NAAF50'] = peaks378_AA50['Q'] * peaks378_50['NAAF factor']
peaks378_AA50['R-NAAF50'] = peaks378_AA50['R'] * peaks378_50['NAAF factor']
peaks378_AA50['S-NAAF50'] = peaks378_AA50['S'] * peaks378_50['NAAF factor']
peaks378_AA50['T-NAAF50'] = peaks378_AA50['T'] * peaks378_50['NAAF factor']
peaks378_AA50['V-NAAF50'] = peaks378_AA50['V'] * peaks378_50['NAAF factor']
peaks378_AA50['W-NAAF50'] = peaks378_AA50['W'] * peaks378_50['NAAF factor']
peaks378_AA50['Y-NAAF50'] = peaks378_AA50['Y'] * peaks378_50['NAAF factor']

# multiply the NAAF50 factor by the PTMs normalize its abundance by peak area and peptide length

peaks378_AA50['ccarb-NAAF50'] = peaks378_AA50['c-carb'] * peaks378_AA50['NAAF factor']
peaks378_AA50['moxid-NAAF50'] = peaks378_AA50['m-oxid'] * peaks378_AA50['NAAF factor']
peaks378_AA50['ndeam-NAAF50'] = peaks378_AA50['n-deam'] * peaks378_AA50['NAAF factor']
peaks378_AA50['qdeam-NAAF50'] = peaks378_AA50['q-deam'] * peaks378_AA50['NAAF factor']
peaks378_AA50['khydr-NAAF50'] = peaks378_AA50['k-hydr'] * peaks378_AA50['NAAF factor']
peaks378_AA50['phydr-NAAF50'] = peaks378_AA50['p-hydr'] * peaks378_AA50['NAAF factor']
peaks378_AA50['rmeth-NAAF50'] = peaks378_AA50['r-meth'] * peaks378_AA50['NAAF factor']

# write the dataframe to a new csv
peaks378_AA50.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-378-100m-trap_PTMopt_DN50_AA_NAAF.csv")

peaks378_AA50.head()

,stripped peptide,NAAF factor,A,C,D,E,F,G,H,K,...,V-NAAF50,W-NAAF50,Y-NAAF50,ccarb-NAAF50,moxid-NAAF50,ndeam-NAAF50,qdeam-NAAF50,khydr-NAAF50,phydr-NAAF50,rmeth-NAAF50
0,VVEEAPK,0.625489,1,0,0,2,0,0,0,1,...,1.250978,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,SLEAESR,0.401095,1,0,0,2,0,0,0,0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,EEDLER,0.223482,0,0,1,3,0,0,0,0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,TEEAVEEK,0.158718,1,0,0,4,0,0,0,1,...,0.158718,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,ELAGNAAR,0.331802,3,0,0,1,0,1,0,0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Same process but for de novo peptide >80 % ALC:

In [7]:
# use a count function to enumerate the # of A's (alanines) in each peptide
peaks378_80['A'] = peaks378_80['Peptide'].str.count("A")

# use a count function to enumerate the # of C's (cysteines) in each peptide
peaks378_80['C'] = peaks378_80['Peptide'].str.count("C")

# use a count function to enumerate the # of D's (aspartic acids) in each peptide
peaks378_80['D'] = peaks378_80['Peptide'].str.count("D")

# use a count function to enumerate the # of E's (glutamic acids) in each peptide
peaks378_80['E'] = peaks378_80['Peptide'].str.count("E")

# use a count function to enumerate the # of F's (phenylalanines) in each peptide
peaks378_80['F'] = peaks378_80['Peptide'].str.count("F")

# use a count function to enumerate the # of G's (glycines) in each peptide
peaks378_80['G'] = peaks378_80['Peptide'].str.count("G")

# use a count function to enumerate the # of H's (histidines) in each peptide
peaks378_80['H'] = peaks378_80['Peptide'].str.count("H")

# use a count function to enumerate the # of I's (isoleucines) in each peptide
# in peaks378_80 output, there will be no isoleucines (they're lumped in with leucines)
peaks378_80['I'] = peaks378_80['Peptide'].str.count("I")

# use a count function to enumerate the # of K's (lysines) in each peptide
peaks378_80['K'] = peaks378_80['Peptide'].str.count("K")

# use a count function to enumerate the # of L's (leucines) in each peptide
# also these include the isoleucines
peaks378_80['I/L'] = peaks378_80['Peptide'].str.count("L")

# use a count function to enumerate the # of M's (methionines) in each peptide
peaks378_80['M'] = peaks378_80['Peptide'].str.count("M")

# use a count function to enumerate the # of N's (asparagines) in each peptide
peaks378_80['N'] = peaks378_80['Peptide'].str.count("N")

# use a count function to enumerate the # of P's ([prolines]) in each peptide
peaks378_80['P'] = peaks378_80['Peptide'].str.count("P")

# use a count function to enumerate the # of Q's (glutamines) in each peptide
peaks378_80['Q'] = peaks378_80['Peptide'].str.count("Q")

# use a count function to enumerate the # of R's (arginines) in each peptide
peaks378_80['R'] = peaks378_80['Peptide'].str.count("R")

# use a count function to enumerate the # of S's (serines) in each peptide
peaks378_80['S'] = peaks378_80['Peptide'].str.count("S")

# use a count function to enumerate the # of T's (threonines) in each peptide
peaks378_80['T'] = peaks378_80['Peptide'].str.count("T")

# use a count function to enumerate the # of V's (valines) in each peptide
peaks378_80['V'] = peaks378_80['Peptide'].str.count("V")

# use a count function to enumerate the # of W's (tryptophans) in each peptide
peaks378_80['W'] = peaks378_80['Peptide'].str.count("W")

# use a count function to enumerate the # of Y's (tyrosines) in each peptide
peaks378_80['Y'] = peaks378_80['Peptide'].str.count("Y")

# use a count function to enumerate the # of carbamidomethylated C's in each peptide
peaks378_80['c-carb'] = peaks378_80['Peptide'].str.count("57.02")

# use a count function to enumerate the # of oxidized M's in each peptide
peaks378_80['m-oxid'] = peaks378_80['Peptide'].apply(lambda x: x.count('M(+15.99)'))

# use a lamba function to enumerate the # of deamidated N's in each peptide
peaks378_80['n-deam'] = peaks378_80['Peptide'].apply(lambda x: x.count('N(+.98)'))

# use a count function to enumerate the # of deamidated Q's in each peptide
peaks378_80['q-deam'] = peaks378_80['Peptide'].apply(lambda x: x.count('Q(+.98)'))

# use a count function to enumerate the # of hydroxylated K's in each peptide
peaks378_80['k-hydr'] = peaks378_80['Peptide'].apply(lambda x: x.count('K(+15.99)'))

# use a count function to enumerate the # of hydroxylated P's in each peptide
peaks378_80['p-hydr'] = peaks378_80['Peptide'].apply(lambda x: x.count('P(+15.99)'))

# use a count function to enumerate the # of methylated R's in each peptide
peaks378_80['r-meth'] = peaks378_80['Peptide'].apply(lambda x: x.count('R(+14.02)'))

# create a column with 'stripped' peptide sequences using strip
peaks378_80['stripped peptide'] = peaks378_80['Peptide'].str.replace(r"\(.*\)","")

# add a column with the stripped peptide length (number of AAs)
peaks378_80['stripped length'] = peaks378_80['stripped peptide'].apply(len)

# total the number of modifications in sequence
peaks378_80['ptm-total'] = peaks378_80['c-carb'] + peaks378_80['m-oxid'] + peaks378_80['n-deam'] + peaks378_80['q-deam'] + peaks378_80['k-hydr'] + peaks378_80['p-hydr'] + peaks378_80['r-meth']

# calculate NAAF numerator for each peptide k
peaks378_80['NAAF num.'] = peaks378_80['Area'] / peaks378_80['stripped length']

# write modified dataframe to new csv file
peaks378_80.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-378-100m-trap_PTMopt_DN80.csv")

# check out the results
peaks378_80.head()

,Fraction,Scan,Source File,Peptide,Tag Length,ALC (%),length,m/z,z,RT,...,m-oxid,n-deam,q-deam,k-hydr,p-hydr,r-meth,stripped peptide,stripped length,ptm-total,NAAF num.
0,4,4564,20181214_378_etnp2017_100m_trap.raw,VVEEAPK,7,99,7,386.2155,2,17.85,...,0,0,0,0,0,0,VVEEAPK,7,0,114285.714286
1,4,4574,20181214_378_etnp2017_100m_trap.raw,SLEAESR,7,98,7,396.1982,2,17.88,...,0,0,0,0,0,0,SLEAESR,7,0,73285.714286
2,4,4982,20181214_378_etnp2017_100m_trap.raw,EEDLER,6,98,6,395.6826,2,19.14,...,0,0,0,0,0,0,EEDLER,6,0,40833.333333
3,4,3812,20181214_378_etnp2017_100m_trap.raw,TEEAVEEK,8,98,8,467.7217,2,15.60,...,0,0,0,0,0,0,TEEAVEEK,8,0,29000.000000
4,4,3992,20181214_378_etnp2017_100m_trap.raw,ELAGNAAR,8,97,8,401.2144,2,16.13,...,0,0,0,0,0,0,ELAGNAAR,8,0,60625.000000


In [8]:
# made a new dataframe that contains the sums of certain columns in modified 
#peptide dataframe above (for >80% ALC)

index = ['sample total']

data = {'A': peaks378_80['A'].sum(),
        'C': peaks378_80['C'].sum(),
        'D': peaks378_80['D'].sum(),
        'E': peaks378_80['E'].sum(),
        'F': peaks378_80['F'].sum(),
        'G': peaks378_80['G'].sum(),
        'H': peaks378_80['H'].sum(),
        'I': peaks378_80['I'].sum(),
        'K': peaks378_80['K'].sum(),
        'I/L': peaks378_80['I/L'].sum(),
        'M': peaks378_80['M'].sum(),
        'N': peaks378_80['N'].sum(),
        'P': peaks378_80['P'].sum(),
        'Q': peaks378_80['Q'].sum(),
        'R': peaks378_80['R'].sum(),
        'S': peaks378_80['S'].sum(),
        'T': peaks378_80['T'].sum(),
        'V': peaks378_80['V'].sum(),
        'W': peaks378_80['W'].sum(),
        'Y': peaks378_80['Y'].sum(),
        'c-carb': peaks378_80['c-carb'].sum(),
        'm-oxid': peaks378_80['m-oxid'].sum(),
        'n-deam': peaks378_80['n-deam'].sum(),
        'q-deam': peaks378_80['q-deam'].sum(),
        'k-hydr': peaks378_80['k-hydr'].sum(),
        'p-hydr': peaks378_80['p-hydr'].sum(),
        'r-meth': peaks378_80['r-meth'].sum(),
        'Total area': peaks378_80['Area'].sum(),
        'Total length': peaks378_80['stripped length'].sum()
       }

totalpeaks378_80 = pd.DataFrame(data, columns=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'I/L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', 'c-carb', 'm-oxid', 'n-deam', 'q-deam', 'k-hydr', 'p-hydr', 'r-meth', 'Total area', 'Total length'], index=index)

# calculate percentage of C's with carb (should be 1.0)
totalpeaks378_80['% C w/ carb'] = totalpeaks378_80['c-carb'] / totalpeaks378_80['C'] 

# calculate percentage of M's that are oxidized
totalpeaks378_80['% M w/ oxid'] = totalpeaks378_80['m-oxid'] / totalpeaks378_80['M'] 

# calculate percentage of N's that are deamidated
totalpeaks378_80['% N w/ deam'] = totalpeaks378_80['n-deam'] / totalpeaks378_80['N'] 

# calculate percentage of Q's that are deamidated
totalpeaks378_80['% Q w/ deam'] = totalpeaks378_80['q-deam'] / totalpeaks378_80['Q'] 

# calculate percentage of K's that are hydroxylated
totalpeaks378_80['% K w/ hydr'] = totalpeaks378_80['k-hydr'] / totalpeaks378_80['K'] 

# calculate percentage of P's that are hydroxylated
totalpeaks378_80['% P w/ hydr'] = totalpeaks378_80['p-hydr'] / totalpeaks378_80['K'] 

# calculate percentage of R's that are methylated
totalpeaks378_80['% R w/ meth'] = totalpeaks378_80['r-meth'] / totalpeaks378_80['R'] 

# calculate NAAF denominator for all peptides in dataset i
totalpeaks378_80['NAAF denom.'] = totalpeaks378_80['Total area'] / totalpeaks378_80['Total length']

# write modified dataframe to new txt file
totalpeaks378_80.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-378-100m-trap_PTMopt_DN80_totals.csv")

totalpeaks378_80.head()

,A,C,D,E,F,G,H,I,K,I/L,...,Total area,Total length,% C w/ carb,% M w/ oxid,% N w/ deam,% Q w/ deam,% K w/ hydr,% P w/ hydr,% R w/ meth,NAAF denom.
sample total,351,23,213,465,86,146,14,0,339,343,...,1.512722e+09,3394,1.0,0.520548,0.294118,0.116279,0.085546,0.235988,0.278626,445704.681791


In [16]:
# use the calculated NAAF factor (in totalpeaks378 dataframe, above) to caluclate the NAAF 
# NAAF: normalized normalized area abundance factor

# don't have to worry here about DECOY hits messing with Area totals
# but we will with Comet results

NAAF80 = 445704.681791

# use NAAF >80% ALC to get NAAF
peaks378_80['NAAF factor'] = (peaks378_80['NAAF num.'])/NAAF80

# make a dataframe that contains only what we need: sequences, AAs, PTMs
peaks378_AA80 = peaks378_80[['stripped peptide', 'NAAF factor', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'K', 'I/L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', 'c-carb', 'm-oxid', 'n-deam', 'q-deam', 'k-hydr', 'p-hydr', 'r-meth']].copy()

# multiply the NAAF80 factor by the AAs to normalize its abundance by peak area and peptide length

peaks378_AA80['A-NAAF80'] = peaks378_AA80['A'] * peaks378_80['NAAF factor']
peaks378_AA80['C-NAAF80'] = peaks378_AA80['C'] * peaks378_80['NAAF factor']
peaks378_AA80['D-NAAF80'] = peaks378_AA80['D'] * peaks378_80['NAAF factor']
peaks378_AA80['E-NAAF80'] = peaks378_AA80['E'] * peaks378_80['NAAF factor']
peaks378_AA80['F-NAAF80'] = peaks378_AA80['F'] * peaks378_80['NAAF factor']
peaks378_AA80['G-NAAF80'] = peaks378_AA80['G'] * peaks378_80['NAAF factor']
peaks378_AA80['H-NAAF80'] = peaks378_AA80['H'] * peaks378_80['NAAF factor']
peaks378_AA80['K-NAAF80'] = peaks378_AA80['K'] * peaks378_80['NAAF factor']
peaks378_AA80['I/L-NAAF80'] = peaks378_AA80['I/L'] * peaks378_80['NAAF factor']
peaks378_AA80['M-NAAF80'] = peaks378_AA80['M'] * peaks378_80['NAAF factor']
peaks378_AA80['N-NAAF80'] = peaks378_AA80['N'] * peaks378_80['NAAF factor']
peaks378_AA80['P-NAAF80'] = peaks378_AA80['P'] * peaks378_80['NAAF factor']
peaks378_AA80['Q-NAAF80'] = peaks378_AA80['Q'] * peaks378_80['NAAF factor']
peaks378_AA80['R-NAAF80'] = peaks378_AA80['R'] * peaks378_80['NAAF factor']
peaks378_AA80['S-NAAF80'] = peaks378_AA80['S'] * peaks378_80['NAAF factor']
peaks378_AA80['T-NAAF80'] = peaks378_AA80['T'] * peaks378_80['NAAF factor']
peaks378_AA80['V-NAAF80'] = peaks378_AA80['V'] * peaks378_80['NAAF factor']
peaks378_AA80['W-NAAF80'] = peaks378_AA80['W'] * peaks378_80['NAAF factor']
peaks378_AA80['Y-NAAF80'] = peaks378_AA80['Y'] * peaks378_80['NAAF factor']

# multiply the NAAF80 factor by the PTMs normalize its abundance by peak area and peptide length

peaks378_AA80['ccarb-NAAF80'] = peaks378_AA80['c-carb'] * peaks378_AA80['NAAF factor']
peaks378_AA80['moxid-NAAF80'] = peaks378_AA80['m-oxid'] * peaks378_AA80['NAAF factor']
peaks378_AA80['ndeam-NAAF80'] = peaks378_AA80['n-deam'] * peaks378_AA80['NAAF factor']
peaks378_AA80['qdeam-NAAF80'] = peaks378_AA80['q-deam'] * peaks378_AA80['NAAF factor']
peaks378_AA80['khydr-NAAF80'] = peaks378_AA80['k-hydr'] * peaks378_AA80['NAAF factor']
peaks378_AA80['phydr-NAAF80'] = peaks378_AA80['p-hydr'] * peaks378_AA80['NAAF factor']
peaks378_AA80['rmeth-NAAF80'] = peaks378_AA80['r-meth'] * peaks378_AA80['NAAF factor']

# write the dataframe to a new csv
peaks378_AA80.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-378-100m-trap_PTMopt_DN80_AA_NAAF.csv")

peaks378_AA80.head()

,stripped peptide,NAAF factor,A,C,D,E,F,G,H,K,...,V-NAAF80,W-NAAF80,Y-NAAF80,ccarb-NAAF80,moxid-NAAF80,ndeam-NAAF80,qdeam-NAAF80,khydr-NAAF80,phydr-NAAF80,rmeth-NAAF80
0,VVEEAPK,0.256416,1,0,0,2,0,0,0,1,...,0.512832,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,SLEAESR,0.164427,1,0,0,2,0,0,0,0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,EEDLER,0.091615,0,0,1,3,0,0,0,0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,TEEAVEEK,0.065066,1,0,0,4,0,0,0,1,...,0.065066,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,ELAGNAAR,0.136021,3,0,0,1,0,1,0,0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Visualizing the results

In [ ]:
print("ALC max: ", peaks['ALC (%)'].max())
print("ALC min: ", peaks['ALC (%)'].min())

In [ ]:
# take only AA totals and transpose for easier bar plotting in matplotlib

peaksaatot = totalpeaks[['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']].copy().T

# take only AA %s and transpose for easier bar plotting in matplotlib

peaksreltot = totalpeaks[['% C w/ carb.', '% M w/ oxid', '% N w/ deam', '% Q w/ deam', '% K w/ hydr', '% P w/ hydr', '% K w/ meth', '% R w/ meth']].copy().T

In [ ]:
# bar plot of residue totals
# there is no isoleucine (I) in Peaks data, which is why L is really big and I is 0


x_labels = ['sample total']

ax = totalpeaks.plot(y=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y'], kind="bar", title = '100 m trap')
plt.xticks(rotation=0)
ax.get_legend().remove()
ax.set_xticklabels(x_labels)

In [ ]:
# bar plot of residue totals
# there is no isoleucine (I) in Peaks data, which is why L is really big and I is 0

my_colors = [(x/10.0, x/20.0, 0.75) for x in range(len(peaksaatot))] # <-- Quick gradient example along the Red/Green dimensions.

ax = peaksaatot.plot(y=['sample total'], kind="bar", color = 'green', title = '100 m trap')


In [ ]:
# bar plot of relative modifications

ax = totalpeaks.plot(y=['% C w/ carb.', '% M w/ oxid', '% N w/ deam', '% Q w/ deam', '% K w/ hydr', '% P w/ hydr', '% K w/ meth', '% R w/ meth'], kind="bar", title = '100 m trap')
ax.set_xticklabels([])

In [ ]:
# bar plot of relative mods


ax = peaksreltot.plot(y=['sample total'], kind="bar", title = '100 m trap')
plt.xticks(rotation=45)

In [ ]:
# making evenly spaced bins for the ALC data based on the min and max, called above
bins = [50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100]
labels = ['50-55', '55-60', '60-65', '65-70', '70-75', '75-80', '80-85', '85-90', '90-95', '95-100']

# use pandas cut function to do the binning itself
peaks['binned'] = pd.cut(peaks['ALC (%)'], bins=bins, labels=labels)

# bar plots of binned PTM data

index = ['50-55', '55-60', '60-65', '65-70', '70-75', '75-80', '80-85', '85-90', '90-95', '95-100']
data = {'Total PTMs': [peaks.groupby('binned')['ptm-total'].sum()['50-55'], peaks.groupby('binned')['ptm-total'].sum()['55-60'], peaks.groupby('binned')['ptm-total'].sum()['60-65'], peaks.groupby('binned')['ptm-total'].sum()['65-70'], peaks.groupby('binned')['ptm-total'].sum()['70-75'], peaks.groupby('binned')['ptm-total'].sum()['75-80'], peaks.groupby('binned')['ptm-total'].sum()['80-85'], peaks.groupby('binned')['ptm-total'].sum()['85-90'], peaks.groupby('binned')['ptm-total'].sum()['90-95'], peaks.groupby('binned')['ptm-total'].sum()['95-100']],
        'Cys carb.': [peaks.groupby('binned')['c-carb'].sum()['50-55'], peaks.groupby('binned')['c-carb'].sum()['55-60'], peaks.groupby('binned')['c-carb'].sum()['60-65'], peaks.groupby('binned')['c-carb'].sum()['65-70'], peaks.groupby('binned')['c-carb'].sum()['70-75'], peaks.groupby('binned')['c-carb'].sum()['75-80'], peaks.groupby('binned')['c-carb'].sum()['80-85'], peaks.groupby('binned')['c-carb'].sum()['85-90'], peaks.groupby('binned')['c-carb'].sum()['90-95'], peaks.groupby('binned')['c-carb'].sum()['95-100']],
        'Met oxi.': [peaks.groupby('binned')['m-oxid'].sum()['50-55'], peaks.groupby('binned')['m-oxid'].sum()['55-60'], peaks.groupby('binned')['m-oxid'].sum()['60-65'], peaks.groupby('binned')['m-oxid'].sum()['65-70'], peaks.groupby('binned')['m-oxid'].sum()['70-75'], peaks.groupby('binned')['m-oxid'].sum()['75-80'], peaks.groupby('binned')['m-oxid'].sum()['80-85'], peaks.groupby('binned')['m-oxid'].sum()['85-90'], peaks.groupby('binned')['m-oxid'].sum()['90-95'], peaks.groupby('binned')['m-oxid'].sum()['95-100']],
        'Asp deam.': [peaks.groupby('binned')['n-deam'].sum()['50-55'], peaks.groupby('binned')['n-deam'].sum()['55-60'], peaks.groupby('binned')['n-deam'].sum()['60-65'], peaks.groupby('binned')['n-deam'].sum()['65-70'], peaks.groupby('binned')['n-deam'].sum()['70-75'], peaks.groupby('binned')['n-deam'].sum()['75-80'], peaks.groupby('binned')['n-deam'].sum()['80-85'], peaks.groupby('binned')['n-deam'].sum()['85-90'], peaks.groupby('binned')['n-deam'].sum()['90-95'], peaks.groupby('binned')['n-deam'].sum()['95-100']],
        'Glut deam.': [peaks.groupby('binned')['q-deam'].sum()['50-55'], peaks.groupby('binned')['q-deam'].sum()['55-60'], peaks.groupby('binned')['q-deam'].sum()['60-65'], peaks.groupby('binned')['q-deam'].sum()['65-70'], peaks.groupby('binned')['q-deam'].sum()['70-75'], peaks.groupby('binned')['q-deam'].sum()['75-80'], peaks.groupby('binned')['q-deam'].sum()['80-85'], peaks.groupby('binned')['q-deam'].sum()['85-90'], peaks.groupby('binned')['q-deam'].sum()['90-95'], peaks.groupby('binned')['q-deam'].sum()['95-100']],
        'Lys hydr': [peaks.groupby('binned')['k-hydr'].sum()['50-55'], peaks.groupby('binned')['k-hydr'].sum()['55-60'], peaks.groupby('binned')['k-hydr'].sum()['60-65'], peaks.groupby('binned')['k-hydr'].sum()['65-70'], peaks.groupby('binned')['k-hydr'].sum()['70-75'], peaks.groupby('binned')['k-hydr'].sum()['75-80'], peaks.groupby('binned')['k-hydr'].sum()['80-85'], peaks.groupby('binned')['k-hydr'].sum()['85-90'], peaks.groupby('binned')['k-hydr'].sum()['90-95'], peaks.groupby('binned')['k-hydr'].sum()['95-100']],
        'Pro hydr': [peaks.groupby('binned')['p-hydr'].sum()['50-55'], peaks.groupby('binned')['p-hydr'].sum()['55-60'], peaks.groupby('binned')['p-hydr'].sum()['60-65'], peaks.groupby('binned')['p-hydr'].sum()['65-70'], peaks.groupby('binned')['p-hydr'].sum()['70-75'], peaks.groupby('binned')['p-hydr'].sum()['75-80'], peaks.groupby('binned')['p-hydr'].sum()['80-85'], peaks.groupby('binned')['p-hydr'].sum()['85-90'], peaks.groupby('binned')['p-hydr'].sum()['90-95'], peaks.groupby('binned')['p-hydr'].sum()['95-100']],
        'Lys meth.': [peaks.groupby('binned')['k-meth'].sum()['50-55'], peaks.groupby('binned')['k-meth'].sum()['55-60'], peaks.groupby('binned')['k-meth'].sum()['60-65'], peaks.groupby('binned')['k-meth'].sum()['65-70'], peaks.groupby('binned')['k-meth'].sum()['70-75'], peaks.groupby('binned')['k-meth'].sum()['75-80'], peaks.groupby('binned')['k-meth'].sum()['80-85'], peaks.groupby('binned')['k-meth'].sum()['85-90'], peaks.groupby('binned')['k-meth'].sum()['90-95'], peaks.groupby('binned')['k-meth'].sum()['95-100']],
        'Arg meth.': [peaks.groupby('binned')['r-meth'].sum()['50-55'], peaks.groupby('binned')['r-meth'].sum()['55-60'], peaks.groupby('binned')['r-meth'].sum()['60-65'], peaks.groupby('binned')['r-meth'].sum()['65-70'], peaks.groupby('binned')['r-meth'].sum()['70-75'], peaks.groupby('binned')['r-meth'].sum()['75-80'], peaks.groupby('binned')['r-meth'].sum()['80-85'], peaks.groupby('binned')['r-meth'].sum()['85-90'], peaks.groupby('binned')['r-meth'].sum()['90-95'], peaks.groupby('binned')['r-meth'].sum()['95-100']]
        }

peaksbin = pd.DataFrame(data, columns=['Total PTMs','Cys carb.','Met oxi.','Asp deam.', 'Glut deam.', 'Lys hydr', 'Pro hydr', 'Lys meth.', 'Arg meth.'], index=index)

# write the peaks bin ptm dataframe to a csv:
peaksbin.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-378-100m-trap_DN50_ptm.csv")

ax1 = peaksbin.plot.bar(y='Total PTMs', rot=45)
ax1.set_title('Total PTMs')

ax2 = peaksbin.plot.bar(y='Cys carb.', rot=45)
ax2.set_title('Cysteine carbamidomethylation')

ax3 = peaksbin.plot.bar(y='Met oxi.', rot=45)
ax3.set_title('Methionine oxidation')

ax4 = peaksbin.plot.bar(y='Asp deam.', rot=45)
ax4.set_title('Asparagine deamidation')

ax5 = peaksbin.plot.bar(y='Glut deam.', rot=45)
ax5.set_title('Glutamine deamidation')

ax6 = peaksbin.plot.bar(y='Lys hydr', rot=45)
ax6.set_title('Lysine hydroxylation')

ax7 = peaksbin.plot.bar(y='Pro hydr', rot=45)
ax7.set_title('Proline hydroxylation')

ax8 = peaksbin.plot.bar(y='Lys meth.', rot=45)
ax8.set_title('Lysine methylation')

ax9 = peaksbin.plot.bar(y='Arg meth.', rot=45)
ax9.set_title('Arginine methylation')


In [ ]:
peaksbin['Total PTMs'] = pd.to_numeric(peaksbin['Total PTMs'])

peaksbin.loc['sums'] = peaksbin.sum(numeric_only=True, axis=0)

peaksbin.loc['rel'] = peaksbin.div(2)['sums']

print(peaksbin)

In [ ]:
# make a new dataframe of relative PTM occurance

# fist pass as numeric
peaks['ptm-total', 'c-carb'] = pd.to_numeric(peaks['ptm-total', 'c-carb'])

data = {'1': [peaks['ptm-total'].sum()], '2': [peaks['c-carb'].sum()]}
#data = {peaks[['ptm-total', 'c-carb', 'm-oxid', 'n-deam', 'q-deam', 'k-hydr', 'p-hydr', 'k-meth', 'r-meth' ]].sum()}

ptmrel = pd.DataFrame(data, columns = ['1', '2'])

ptmrel.head()

### Exporting txt files of stripped peptides at confidence cutoffs:

In [ ]:
# keep only peptide column >50% ALC
pep = peaks[["stripped peptide"]]

# write altered dataframe to new txt file
# used header and index parameters to get rid of 'Peptide' header and the indexing

pep.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-378-100m-trap_DN50_stripped_peptides.txt", header=False, index=False)


# look

print("# of DN peptide >50% ALC", len(pep))
pep.head()

In [ ]:
# keep only peptides  >80% ALC
peaks80 = peaks.loc[peaks['ALC (%)'] >= 80]

# see how many rows and double check
# peaks80.head(-10)

# keep only peptide column 
pep80 = peaks80[["stripped peptide"]]

# write altered dataframe to new txt file
# used header and index parameters to get rid of 'Peptide' header and the indexing

pep80.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-378-100m-trap_DN80_stripped_peptides.txt", header=False, index=False)


print("# of DN peptide >80% ALC", len(pep80))
pep80.head()